3. Faça uma tabela com média e desvio-padrão das variáveis. Faça outra tabela com a correlação entre S e IQ.

In [1]:
import pandas as pd

df = pd.read_excel('grilic.xls')


In [2]:
parametros = df.agg(['mean', 'std'], numeric_only=True)

correlacao_S_IQ = df['S'].corr(df['IQ'])


4. Apresente uma análise da escolha dos instrumentos para resolver os problemas de viés de variável omitida e erro-nas-variáveis. Inclua nesta análise a regressão de primeiro estágio que achar pertinente.

Instrumentos usados poderiam ser KWW, MED

Os instrumentos devem ser correlacionados com as variáveis endógenas (IQ e S) e devem ser exógenos

In [6]:
linhas_interesse = ['S', 'IQ']
colunas_interesse = ['MED', 'KWW', 'MRT', 'AGE']

# Calcular a matriz de correlação completa para todas as colunas relevantes
# Primeiro, criamos uma lista com todas as colunas que participarão
colunas_para_corr_completa = list(set(linhas_interesse + colunas_interesse)) # set() para evitar duplicatas
matriz_corr_completa = df[colunas_para_corr_completa].corr(numeric_only=True)

# Selecionar apenas as correlações de interesse
matriz_corr_especifica = matriz_corr_completa.loc[linhas_interesse, colunas_interesse]

correlacao_iv = pd.DataFrame(matriz_corr_especifica).T

In [ ]:
import statsmodels.api as sm

year_column = 'YEAR' # << AJUSTE AQUI SE O NOME DA COLUNA DE ANO FOR DIFERENTE (ex: 'YEAR')


year_dummies = pd.get_dummies(df[year_column], prefix=year_column, drop_first=True, dtype=int)

# Juntar as dummies ao DataFrame (ou a uma cópia para não modificar o original)
df_with_dummies = pd.concat([df, year_dummies], axis=1)

dependent_var = 'S'
original_independent_vars = ['RNS', 'SMSA', 'MED', 'KWW', 'MRT', 'AGE']
# Adicionar os nomes das colunas dummy à lista de variáveis independentes
all_independent_vars = original_independent_vars + year_dummies.columns.tolist()

# Verificar se todas as colunas necessárias existem no df_with_dummies
# (Verifica no df_with_dummies, que agora contém as dummies e as originais)
missing_cols_check = [col for col in all_independent_vars if col not in df_with_dummies.columns]
if dependent_var not in df_with_dummies.columns: # Checa também a variável dependente
missing_cols_check.append(dependent_var)


Y = df_with_dummies[dependent_var].astype(float) # Garante que Y seja float
X = df_with_dummies[all_independent_vars].astype(float) # Garante que X seja float
X = sm.add_constant(X) # Adicionar o intercepto

# Lidar com NaNs combinando Y e X e depois removendo linhas com qualquer NaN
# Isso garante que Y e X tenham o mesmo número de observações válidas.
combined_data = pd.concat([Y, X], axis=1).dropna()

if combined_data.shape[0] < X.shape[1]: # Checar se há dados suficientes após dropna
    print(f"Dados insuficientes para realizar a regressão após remover NaNs.")
    print(f"Observações restantes: {combined_data.shape[0]}, Regressores (incl. intercepto): {X.shape[1]}")
else:
    Y_clean = combined_data[dependent_var]
    # X_clean deve ser selecionado do combined_data para manter o alinhamento após dropna
    # e para garantir que contenha as colunas corretas (incluindo 'const' e as dummies)
    X_clean = combined_data.drop(columns=[dependent_var]) 
    
    model = sm.OLS(Y_clean, X_clean)
    results = model.fit()
    print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      S   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.488
Method:                 Least Squares   F-statistic:                     61.11
Date:                Thu, 12 Jun 2025   Prob (F-statistic):          2.58e-102
Time:                        19:10:17   Log-Likelihood:                -1423.9
No. Observations:                 758   AIC:                             2874.
Df Residuals:                     745   BIC:                             2934.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.9929      0.563      7.094      0.0

In [ ]:
dependent_var = 'IQ'
original_independent_vars = ['RNS', 'SMSA', 'MED', 'KWW', 'MRT', 'AGE']

missing_cols_check.append(dependent_var)


Y = df_with_dummies[dependent_var].astype(float) # Garante que Y seja float
X = df_with_dummies[all_independent_vars].astype(float) # Garante que X seja float
X = sm.add_constant(X) # Adicionar o intercepto


combined_data = pd.concat([Y, X], axis=1).dropna()

if combined_data.shape[0] < X.shape[1]: # Checar se há dados suficientes após dropna
    print(f"Dados insuficientes para realizar a regressão após remover NaNs.")
    print(f"Observações restantes: {combined_data.shape[0]}, Regressores (incl. intercepto): {X.shape[1]}")
else:
    Y_clean = combined_data[dependent_var]
    # X_clean deve ser selecionado do combined_data para manter o alinhamento após dropna
    # e para garantir que contenha as colunas corretas (incluindo 'const' e as dummies)
    X_clean = combined_data.drop(columns=[dependent_var]) 
    
    model = sm.OLS(Y_clean, X_clean)
    results = model.fit()
    print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     IQ   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     20.82
Date:                Thu, 12 Jun 2025   Prob (F-statistic):           1.06e-39
Time:                        19:14:29   Log-Likelihood:                -2944.9
No. Observations:                 758   AIC:                             5916.
Df Residuals:                     745   BIC:                             5976.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         76.0966      4.187     18.176      0.0

5. Griliches também argumenta que escolaridade pode ser endógena. Qual o argumento dele? Estime a equação salarial por 2SLS assumindo IQ e S como endógenas. O que acontece com o coeciente de escolaridade? Como você pode explicar a diferença entre sua estimativa 2SLS de S e a estimativa anterior (sem tratar S como endógena)? Calcule a estatística de Sargan. 